In [12]:
import pandas as pd
from datetime import datetime, timedelta
from prettytable import PrettyTable
import random

# Read data from CSV files
data = pd.read_csv("2008.csv")
carrier = pd.read_csv("AirlineCodes.csv")
airport = pd.read_csv("airports.csv")

# Get current date and time
current_month = datetime.now().month
current_day = datetime.now().day
current_time = datetime.now().time()

# Filter itinerary for current day, get needed columns
itinerary = data.loc[(data['Month'] == current_month) & (data['DayofMonth'] == current_day), ['UniqueCarrier', 'DepTime', 'ArrTime', 'FlightNum', 'Origin', 'Dest']]
itinerary = itinerary.dropna()

# Create mapping dictionaries for carrier codes and airport IDs
carrier_mapping = dict(zip(carrier['Code'], carrier['Name']))
airport_mapping = dict(zip(airport['ident'].str.lstrip('K'), airport['municipality'].str.split('/ ').str[0]))

# Extract delay and cancellation data
delays = data[['DepDelay']]
cancellations = data[['Cancelled']]

# Make a gate number based on FlightNum
def get_gate(FlightNum):
    FlightNum = str(FlightNum)  # Convert to string
    first_digit = int(FlightNum[0])
    last_two_digits = int(FlightNum[-2:])
    
    letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
    gate_letter = letters[first_digit - 1]

    gate_number = first_digit + last_two_digits
    
    if gate_number > 99:
        gate_number = int(gate_number / 1.5)

    return f"{gate_letter}{gate_number}"

# Parse time strings into datetime.time objects
def parse_time(time_str):
    try:
        return datetime.strptime(str(time_str).split('.')[0], "%H%M").time()
    except ValueError:
        pass

# Check if flight has a delay
def has_delay(row): 
    for column in delays:
        if pd.notna(data.loc[row.name, column]) and data.loc[row.name, column] >= 15:
            return True
    return False

# Check if flight is cancelled
def cancelled(row):
    for column in cancellations:
        if pd.notna(data.loc[row.name, column]) and data.loc[row.name, column] != 0:
            return True
    return False

# Check if boarding has begun for flight
def boarding(row):
    dep_time = parse_time(row['DepTime'])
    if dep_time is not None:
        boarding_time = datetime.combine(datetime.today(), dep_time) - timedelta(minutes=45)
        return boarding_time.time() <= current_time <= dep_time
    return False

# Check if flight has departed
def departed(row):
    dep_time = parse_time(row['DepTime'])
    if dep_time is not None:
        push_back = datetime.combine(datetime.today(), dep_time) - timedelta(minutes=10)
        return push_back.time() <= current_time <= dep_time
    return False

# Get airline name based on the carrier code
def get_airline(carrier_code):
    if carrier_code in carrier_mapping:
        return carrier_mapping[carrier_code]
    return "Void"

# Get airport name based on the airport ID
def get_airport(id):
    if id in airport_mapping:
        return airport_mapping[id]
    return "Void"

# Get status of flight
def get_status(row):
    if has_delay(row) and not boarding(row) and not departed(row):
        return "Delayed"
    elif cancelled(row):
        return "Cancelled"
    elif boarding(row) and not departed(row):
        return "Boarding"
    elif departed(row):
        return "Departed"
    else:
        return "On Time"

# Format time objects as strings
def format_time(time):
    return time.strftime("%I:%M %p")

# Generate the arrival board
def generate_arrival_board(rows):
    tableA = PrettyTable()
    tableA.field_names = ["Airline", "Flight", "Origin", "Status", "Time", "Gate"]
    
    for _, row in rows.iterrows():
        arr_time = parse_time(row['ArrTime'])
        dep_time = parse_time(row['DepTime'])
        origin = get_airport(row['Origin'])
        if arr_time is not None and dep_time is not None and origin != 'Void':
            if current_time >= dep_time and current_time <= arr_time: 
                tableA.add_row([
                    get_airline(row['UniqueCarrier']),
                    row['FlightNum'],
                    get_airport(row['Origin']),
                    get_status(row),
                    format_time(arr_time),
                    get_gate(row['FlightNum'])])
    return tableA

# Generate the departure board
def generate_departure_board(rows):
    tableD = PrettyTable()
    tableD.field_names = ["Airline", "Flight", "Destination", "Status", "Time", "Gate"]

    for _, row in rows.iterrows():
        dep_time = parse_time(row['DepTime'])
        destination = get_airport(row['Dest'])
        if dep_time is not None and destination != 'Void':
            if current_time <= dep_time: 
                tableD.add_row([
                    get_airline(row['UniqueCarrier']),
                    row['FlightNum'],
                    get_airport(row['Dest']),
                    get_status(row),
                    format_time(dep_time),
                    get_gate(row['FlightNum'])])
    return tableD

# Display arrival board for specificied airport
def arrivals(airport):
    filtered_rows = itinerary[itinerary['Dest'] == airport]
    return generate_arrival_board(filtered_rows)

# Display departure board for specificied airport
def departures(airport):
    filtered_rows = itinerary[itinerary['Origin'] == airport]
    return generate_departure_board(filtered_rows)

# Create variables
arrival_board = arrivals('ILM')
departure_board = departures('ILM')
display_time = format_time(current_time)

print(display_time)

print("\nArrivals:")
print(arrival_board)

print("\nDepartures:")
print(departure_board)


04:33 PM

Arrivals:
+---------+--------+---------+---------+----------+------+
| Airline | Flight |  Origin |  Status |   Time   | Gate |
+---------+--------+---------+---------+----------+------+
|   PSA   |  5706  | Atlanta | On Time | 05:15 PM | E11  |
+---------+--------+---------+---------+----------+------+

Departures:
+---------+--------+-------------+---------+----------+------+
| Airline | Flight | Destination |  Status |   Time   | Gate |
+---------+--------+-------------+---------+----------+------+
|   Mesa  |  2653  |  Charlotte  | On Time | 07:50 PM | B55  |
|   PSA   |  5349  | Cincinnati  | Delayed | 05:35 PM | E54  |
+---------+--------+-------------+---------+----------+------+
